In [110]:
import pandas as pd
import hmac
import hashlib
import requests
from io import StringIO

In [111]:
with open('../checksums.txt', 'rb') as f:
    data = f.read()
    checksum_verify = str(data.split()[1]).split("'")[1]

url = 'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series/13/pcp/1/9/1895-2024.csv?base_prd=true&begbaseyear=1901&endbaseyear=2000'
response = requests.get(url)
checksum = hashlib.sha256(response.text.encode('utf-8')).hexdigest()

if checksum == checksum_verify:
    print('Checksum verification passed.\n')
    df_ncei = pd.read_csv(StringIO(response.text), skiprows=4)
    print(df_ncei.head())

else:
   print('Checksum verification failed.')

Checksum verification passed.

     Date  Value  Anomaly
0  189509   3.12    -0.55
1  189609   3.97     0.30
2  189709   1.97    -1.70
3  189809   2.62    -1.05
4  189909   0.96    -2.71


In [129]:
with open('../checksums.txt', 'rb') as f:
    data = f.read()
    checksum_verify = str(data.split()[3]).split("'")[1]

endpoint = 'https://data.iowa.gov/resource/tw78-ziwj.csv?$limit=583174'
response = requests.get(endpoint)
checksum = hashlib.sha256(response.text.encode('utf-8')).hexdigest()

if checksum == checksum_verify:
    print('Checksum verification passed.\n')
    df_accidents = pd.read_csv(StringIO(response.text))
    print(df_accidents.head())

else:
   print('Checksum verification failed.')

Checksum verification failed.


In [113]:
# cleaning df_accidents

## converting `date_of_crash` to `year` 
date_time = pd.to_datetime(df_accidents['date_of_crash'])
df_accidents['year'] = date_time.dt.year

## narrowing `df_accidents` to contain only our columns of interest
df_accidents = df_accidents[[
    'year', 'enviro_conditions', 'surface_conditions', 'weather_conditions',
    'crash_severity', 'fatalities', 'injuries', 'majinjury', 'mininjury',
    'possinjury', 'unkinjury', 'propdmg'
]]

In [114]:
print(df_accidents['enviro_conditions'].unique())
print(df_accidents['weather_conditions'].unique())
print(df_accidents['surface_conditions'].unique())
print(df_accidents['crash_severity'].unique())


[nan 'None apparent' 'Unknown' 'Weather conditions' 'Animal in roadway'
 'Glare' 'Other (explain in narrative)' 'Visual obstruction'
 'Non-motorist action' 'Severe crosswind']
['Cloudy' 'Snow' 'Clear' 'Unknown' nan 'Rain'
 'Other (explain in narrative)' 'Sleet, hail' 'Blowing sand, soil, dirt'
 'Severe winds' 'Fog, smoke, smog' 'Freezing rain/drizzle' 'Blowing snow']
['Snow' 'Ice/frost' 'Slush' 'Wet' 'Dry' 'Other (explain in narrative)' nan
 'Unknown' 'Mud, dirt' 'Water (standing or moving)' 'Gravel' 'Sand' 'Oil']
['Minor Injury' 'Property Damage Only' 'Possible/Unknown' 'Major Injury'
 'Fatal']


In [115]:
df_accidents['enviro_conditions'].isna().value_counts()

enviro_conditions
False    436991
True     146183
Name: count, dtype: int64

In [116]:
df_accidents['enviro_conditions'] = df_accidents['enviro_conditions'].fillna('Unknown')
df_accidents['surface_conditions'] = df_accidents['surface_conditions'].fillna('Unknown')

## Want to make sure we get `Rain`,`Sleet, hail`,`Freezing rain/drizzle` within `weather_conditions`

## Want to make sure we get `Wet` within `surface_conditions`

In [117]:
filtered_test = df_accidents[((df_accidents['weather_conditions'] == 'Rain') | (df_accidents['weather_conditions'] == 'Sleet, hail') | (df_accidents['weather_conditions'] == 'Freezing rain/drizzle')) & ((df_accidents['surface_conditions'] == 'Wet') | (df_accidents['surface_conditions'] == 'Unknown'))]
filtered_test.sample(10)

,year,enviro_conditions,surface_conditions,weather_conditions,crash_severity,fatalities,injuries,majinjury,mininjury,possinjury,unkinjury,propdmg
174937,2017,None apparent,Wet,Rain,Possible/Unknown,0,1,0,0,1,0,3000
193298,2017,Weather conditions,Wet,Rain,Property Damage Only,0,0,0,0,0,0,9300
107403,2016,None apparent,Wet,Rain,Property Damage Only,0,0,0,0,0,0,1700
155455,2016,None apparent,Wet,Rain,Property Damage Only,0,0,0,0,0,0,3500
186175,2017,Weather conditions,Wet,Rain,Possible/Unknown,0,1,0,0,1,0,50000
244639,2018,Weather conditions,Wet,Rain,Property Damage Only,0,0,0,0,0,0,15000
50796,2014,Unknown,Wet,Rain,Property Damage Only,0,0,0,0,0,0,4500
295357,2019,None apparent,Wet,Rain,Property Damage Only,0,0,0,0,0,0,16200
23555,2014,Unknown,Wet,Rain,Property Damage Only,0,0,0,0,0,0,3000
292576,2019,None apparent,Wet,Rain,Property Damage Only,0,0,0,0,0,0,3000


In [118]:
df_ncei.head(5)

,Date,Value,Anomaly
0,189509,3.12,-0.55
1,189609,3.97,0.30
2,189709,1.97,-1.70
3,189809,2.62,-1.05
4,189909,0.96,-2.71


In [119]:
## cleaning df_ncei

df_ncei['year'] = df_ncei['Date'].apply(lambda x: int(str(x)[:4]))
df_ncei.rename(columns={'Value':'precipitation'}, inplace=True)
df_ncei = df_ncei[['year', 'precipitation']]
df_ncei['year'] = df_ncei['year'].astype(int)
df_ncei = df_ncei[df_ncei['year'] >= 2014]
df_ncei.head()

,year,precipitation
119,2014,4.35
120,2015,4.18
121,2016,6.12
122,2017,2.31
123,2018,8.10


In [120]:
grouped_df = filtered_test.groupby(['year', 'enviro_conditions', 'surface_conditions', 'weather_conditions', 'crash_severity']).sum().reset_index()
grouped_df


,year,enviro_conditions,surface_conditions,weather_conditions,crash_severity,fatalities,injuries,majinjury,mininjury,possinjury,unkinjury,propdmg
0,2014,Unknown,Unknown,Rain,Possible/Unknown,0,3,0,0,3,0,6300
1,2014,Unknown,Unknown,Rain,Property Damage Only,0,0,0,0,0,0,24300
2,2014,Unknown,Unknown,"Sleet, hail",Possible/Unknown,0,1,0,0,1,0,11500
3,2014,Unknown,Wet,Rain,Fatal,19,21,5,9,7,0,365500
4,2014,Unknown,Wet,Rain,Major Injury,0,114,81,23,10,0,903187
...,...,...,...,...,...,...,...,...,...,...,...,...
463,2024,Weather conditions,Wet,Rain,Minor Injury,0,54,0,44,8,2,496900
464,2024,Weather conditions,Wet,Rain,Possible/Unknown,0,85,0,0,74,11,701358
465,2024,Weather conditions,Wet,Rain,Property Damage Only,0,0,0,0,0,0,2821177
466,2024,Weather conditions,Wet,"Sleet, hail",Possible/Unknown,0,1,0,0,1,0,3000


In [121]:
yearly_counts = {'year': [], 'count': []}

current_year = None
count = 0

for index, row in filtered_test.iterrows():
    if row['year'] == current_year:
        count += 1
    else:
        if current_year is not None:
            yearly_counts['year'].append(current_year)
            yearly_counts['count'].append(count)
        current_year = row['year']
        count = 1

yearly_counts['year'].append(current_year)
yearly_counts['count'].append(count)

yearly_counts_df = pd.DataFrame(yearly_counts)
yearly_counts_df


,year,count
0,2014,3017
1,2015,3986
2,2016,2679
3,2017,3047
4,2018,3389
5,2019,3093
6,2020,2276
7,2021,2383
8,2022,2359
9,2023,2012


In [122]:
grouped_df.head(10)

,year,enviro_conditions,surface_conditions,weather_conditions,crash_severity,fatalities,injuries,majinjury,mininjury,possinjury,unkinjury,propdmg
0,2014,Unknown,Unknown,Rain,Possible/Unknown,0,3,0,0,3,0,6300
1,2014,Unknown,Unknown,Rain,Property Damage Only,0,0,0,0,0,0,24300
2,2014,Unknown,Unknown,"Sleet, hail",Possible/Unknown,0,1,0,0,1,0,11500
3,2014,Unknown,Wet,Rain,Fatal,19,21,5,9,7,0,365500
4,2014,Unknown,Wet,Rain,Major Injury,0,114,81,23,10,0,903187
5,2014,Unknown,Wet,Rain,Minor Injury,0,398,0,337,57,4,2653358
6,2014,Unknown,Wet,Rain,Possible/Unknown,0,638,0,0,601,37,3752727
7,2014,Unknown,Wet,Rain,Property Damage Only,0,0,0,0,0,0,11380753
8,2014,Unknown,Wet,"Sleet, hail",Major Injury,0,2,2,0,0,0,17200
9,2014,Unknown,Wet,"Sleet, hail",Minor Injury,0,11,0,10,0,1,79453


In [123]:
grouped_df_2 = filtered_test.groupby(['year']).sum().reset_index()
grouped_df_2

,year,enviro_conditions,surface_conditions,weather_conditions,crash_severity,fatalities,injuries,majinjury,mininjury,possinjury,unkinjury,propdmg
0,2014,UnknownUnknownUnknownUnknownUnknownUnknownUnkn...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,"RainRainRainRainSleet, hailSleet, hailRainRain...",Property Damage OnlyProperty Damage OnlyProper...,19,1191,88,379,682,42,19511105
1,2015,UnknownUnknownUnknownUnknownUnknownUnknownUnkn...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,Freezing rain/drizzleFreezing rain/drizzleRain...,Property Damage OnlyMinor InjuryProperty Damag...,23,1606,71,460,966,109,26318128
2,2016,None apparentNone apparentNone apparentNone ap...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,Freezing rain/drizzleFreezing rain/drizzleFree...,Possible/UnknownProperty Damage OnlyProperty D...,7,1013,46,277,611,79,17648125
3,2017,Weather conditionsNone apparentNone apparentNo...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,RainRainFreezing rain/drizzleFreezing rain/dri...,Possible/UnknownMajor InjuryProperty Damage On...,9,1190,69,327,692,102,20706389
4,2018,None apparentNone apparentNone apparentWeather...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,Freezing rain/drizzleFreezing rain/drizzleFree...,Property Damage OnlyProperty Damage OnlyProper...,12,1260,51,357,743,109,22543154
5,2019,None apparentWeather conditionsWeather conditi...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,RainFreezing rain/drizzleRainRainRainRainRainR...,Property Damage OnlyProperty Damage OnlyProper...,18,1244,66,355,728,95,21840047
6,2020,Animal in roadwayNone apparentNone apparentNon...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,RainFreezing rain/drizzleRainRainRainFreezing ...,Property Damage OnlyPossible/UnknownProperty D...,17,845,47,232,472,94,17328310
7,2021,Weather conditionsWeather conditionsWeather co...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,Freezing rain/drizzleFreezing rain/drizzleFree...,Possible/UnknownPossible/UnknownProperty Damag...,11,975,41,299,539,96,19965911
8,2022,None apparentWeather conditionsWeather conditi...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,Freezing rain/drizzleFreezing rain/drizzleFree...,Possible/UnknownProperty Damage OnlyPossible/U...,13,914,62,285,469,98,20642149
9,2023,Weather conditionsNone apparentNone apparentWe...,WetWetWetWetWetWetWetWetWetWetWetWetWetWetWetW...,RainRainRainFreezing rain/drizzleRainRainRainR...,Minor InjuryProperty Damage OnlyPossible/Unkno...,11,765,37,263,383,82,18308339


In [124]:
merged_df = pd.merge(df_ncei, grouped_df_2, on='year', how='inner')
merged_df = pd.merge(yearly_counts_df, merged_df, on='year', how='inner')
merged_df = merged_df[['year','precipitation','count','fatalities','injuries','majinjury','mininjury','possinjury','unkinjury','propdmg']]
merged_df

,year,precipitation,count,fatalities,injuries,majinjury,mininjury,possinjury,unkinjury,propdmg
0,2014,4.35,3017,19,1191,88,379,682,42,19511105
1,2015,4.18,3986,23,1606,71,460,966,109,26318128
2,2016,6.12,2679,7,1013,46,277,611,79,17648125
3,2017,2.31,3047,9,1190,69,327,692,102,20706389
4,2018,8.10,3389,12,1260,51,357,743,109,22543154
5,2019,6.21,3093,18,1244,66,355,728,95,21840047
6,2020,4.07,2276,17,845,47,232,472,94,17328310
7,2021,1.81,2383,11,975,41,299,539,96,19965911
8,2022,1.90,2359,13,914,62,285,469,98,20642149
9,2023,2.14,2012,11,765,37,263,383,82,18308339


In [ ]:
merged_df.to_csv('../data/processed/integrated.csv')